In [7]:
import mlflow
mlflow.set_tracking_uri("http://13.221.75.66:5000/")


c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [8]:
# Set experiment before logging anything
mlflow.set_experiment("ML Algos with HPT")

with mlflow.start_run():
    mlflow.log_param("example_param", 42)


2025/08/14 12:38:55 INFO mlflow.tracking.fluent: Experiment with name 'ML Algos with HPT' does not exist. Creating a new experiment.


In [ ]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import mlflow.sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import osimport 

In [ ]:
df= pd.read_csv(r'C:\Users\Deepu\OneDrive\Desktop\Capstone1\reddit_preprocessing.csv').dropna()
df.shape

(36750, 7)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import mlflow
import mlflow.sklearn
import optuna

mlflow.set_experiment("ML Algos with HPT")

# Preprocessing
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})
df = df.dropna(subset=['category'])
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

def objective_rf(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5)
    }
    model = RandomForestClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    return accuracy_score(y_test, model.predict(X_test))

study = optuna.create_study(direction="maximize")
study.optimize(objective_rf, n_trials=30)
best_params = study.best_params

with mlflow.start_run(run_name="RandomForest"):
    best_model = RandomForestClassifier(**best_params, random_state=42)
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    for label, metrics in report.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)
    for param, value in best_params.items():
        mlflow.log_param(param, value)
    
    model_info = mlflow.sklearn.log_model(best_model, artifact_path="best_randomforest_model")
    print("Best Parameters:", best_params)
    print("Model Saved at:", model_info.model_uri)


[I 2025-08-14 12:49:47,939] A new study created in memory with name: no-name-73163a0f-22bc-463d-b73b-af8a55d17bd1
[I 2025-08-14 12:50:04,981] Trial 0 finished with value: 0.7130627774254914 and parameters: {'n_estimators': 200, 'max_depth': 16, 'min_samples_split': 7, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.7130627774254914.
[I 2025-08-14 12:50:09,799] Trial 1 finished with value: 0.6481716339040372 and parameters: {'n_estimators': 262, 'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.7130627774254914.
[I 2025-08-14 12:50:12,078] Trial 2 finished with value: 0.6252377932783767 and parameters: {'n_estimators': 191, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.7130627774254914.
[I 2025-08-14 12:50:21,475] Trial 3 finished with value: 0.7082012259564574 and parameters: {'n_estimators': 180, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 0 with value: 